In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 100)
from matplotlib import rcParams
rcParams['font.family'] = 'Noto Sans CJK JP'
%matplotlib inline

In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

/home/kengo-fujii/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
whole_df = pd.concat([train_df, test_df], axis=0)

In [4]:
whole_df["所在地"] = whole_df["都道府県名"]+whole_df["市区町村名"]+whole_df["地区名"]
geo_df = whole_df.groupby("所在地").count().reset_index()[["所在地", "ID"]]

In [5]:
geo_df

,所在地,ID
0,三重県三重郡川越町大字北福崎,1
1,三重県三重郡川越町大字当新田,5
2,三重県三重郡川越町大字豊田,32
3,三重県三重郡川越町大字高松,5
4,三重県三重郡朝日町大字小向,3
...,...,...
15518,鹿児島県鹿児島市高麗町,51
15519,鹿児島県鹿児島市鴨池,28
15520,鹿児島県鹿児島市鴨池新町,149
15521,鹿児島県鹿屋市大手町,2


In [6]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="test4")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

geo_df['location'] = geo_df['所在地'].apply(geocode, timeout=30000)


KeyboardInterrupt: 

In [19]:
geo_df['latitude'] = geo_df['location'].apply(lambda x:x.point[0] if x else None)
geo_df['longitude'] = geo_df['location'].apply(lambda x:x.point[1] if x else None)

In [21]:
geo_df = geo_df.drop(["ID", "location"],axis=1)

In [29]:
# データの形を綺麗にする
merge_df = pd.merge(geo_df, whole_df[["都道府県名", "市区町村名", "地区名", "所在地"]], on="所在地", how="left").drop_duplicates()

In [31]:
assert len(merge_df)==len(geo_df)

In [34]:
merge_df.to_csv("../add/geo_info.csv", index=False)